In [2]:
from pathlib import Path
import re

In [3]:
def read_wnut(file_path):
    file_path = Path(file_path)
    
    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text) #creates a list and splits based on the pattern \n\t?\n which is the newline, tab, newline that comes after every sample sequence
    token_docs = []
    tag_docs = []
    for doc in raw_docs: #iterates over each sample sequence in list
        tokens = []
        tags = []
        for line in doc.split('\n'): #sequence based on newlines (this breaks it down to each token with its corresponding label per line)
            token, tag = line.split('\t') #splits into a list based on \t (tab) 
            tokens.append(token) #adds the new token to tokens list
            tags.append(tag) #adds the new tag to tags list (both token and tag have the same index value)
        token_docs.append(tokens) #appends the list of tokens in the sequence to masterlist of tokens
        tag_docs.append(tags) #appends the list of tags in the sequence to masterlist of tags
    return token_docs, tag_docs #returns two lists (filled with lists), one of all tokens, one of all tags where the masterlist contains a list of lists where each inner list contains the tokens/tags for one sequence

texts, tags = read_wnut("/mnt/storage/grid/home/eric/hmm2bert/example/token_classification_W-NUT/wnut17train.conll")

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train_text, val_text, train_tags, val_tags = train_test_split(texts, tags, test_size=0.2) #create train-test split

unique_tags = set(tag for doc in tags for tag in doc) #nested for loop where for doc in tags, for tag in docs, tag (create a set containing all the unique tags in the masterlist tags)

#use to find ID of a certain tag
tag2id = {tag: id for id, tag in enumerate(unique_tags)} #enumerate returns a tuple (ID, TAG) so this creates a dict with tags as the key and ID number as the value [for id, tag in enumerate(unique_tags)]

#use to find the label for a certain ID
id2tag = {id: tag for tag, id in tag2id.items()} #.items() returns a list of tuples (TAG, ID) of the dictionary passed [for tag, id in unique_tags.itmes()]



In [6]:
from transformers import DistilBertTokenizerFast

In [7]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

train_encodings = tokenizer(train_text, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_text, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [14]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [18]:
import torch

class WNUTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [19]:
train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)

In [20]:
train_dataset[0]

{'input_ids': tensor([  101,  3949,  1116,  1706,  3969, 12218,  8912,   106,   131,   141,
           131,   141,   131,   149,  2137,   111,   181,  1204,   132,   124,
          2185,   162,  2346,  1358,  9896,   106,   106,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0